In [1]:
#importing modules
import numpy as np
import matplotlib.pyplot as plt
import h5py

from astropy.io import fits as pyfits
from astropy import wcs
from astropy.table import Table, join, vstack
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from scipy.optimize import curve_fit

from scipy.misc import imresize

from random import choice

In [26]:
# Setting stuff
nan= float('nan')

pi= np.pi
e= np.e

plt.close('all')
c= 3.*10**8
h= 6.62*10**-34
k= 1.38*10**-23

Ken98= (4.5*10**-44)
Conversion2Chabrier=1.7 # Also Madau
Calzetti12= 2.8*10**-44
arrow = u'$\u2193$' 


Hal_cent = 6562.8
NIIr_cent = 6582.
NIIb_cent = 6548.

PATH='./'

# Getting the fuctions working 

In [27]:
def gaussian(x, a, b, c):
    val = a * np.exp(-(x - b)**2 / (2.*c**2))
    return val


def save_obj(obj, name ):
    with open(PATH + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    import pickle
    with open(PATH + name + '.pkl', 'rb') as f:
        return pickle.load(f, encoding='latin1')
    

In [34]:
def spec_gen_Halpha():
    # generating random properties
    z = np.random.uniform(1.4, 2.5)
     
    Nar_sig = np.random.uniform(200,700)/2.35   # Width of the narrow components
    Nar_fl = 10**np.random.uniform(-15.3, -12.3) # Flux of narrow Halpha
    NII_fl = np.random.uniform(0.1,1)*Nar_fl # Flux of [NII]
    
    
    Cont = np.random.uniform(0.01,0.2)*Nar_fl
    # Are we doing Type 1 or 2
    Type = choice([0,1])
    
    # Generating BLR proeprties
    if Type==1:
        BLR_fl = np.random.uniform(0.1,1)*Nar_fl
        BLR_sig = np.random.uniform(4000,8000)/2.35  
    else:
         BLR_fl = 0
         BLR_sig = np.random.uniform(4000,8000)/2.35  
        
    ###ll
    # Selecting Which band it was observed based on redshift
    Hw = np.linspace(1.42533, 1.86678, 2048)
    Kw = np.linspace(1.92543, 2.49972, 2048)
    
    if z<1.83:
        x=Hw
    elif z>1.94:
        x=Kw
    else:
        z = np.random.uniform(1.4, 2.5, 1)
        if z<1.83:
            x=Hw
        elif z>1.94:
            x=Kw
    
    Nar_sig_wv = Nar_sig/3e5*(6562.8/1e4*(1+z))
    BLR_sig_wv = BLR_sig/3e5*(6562.8/1e4*(1+z))
    
    
    Nar_comp = gaussian(x, Nar_fl, Hal_cent/1e4*(1+z), Nar_sig_wv) + \
                gaussian(x, NII_fl, NIIr_cent/1e4*(1+z), Nar_sig_wv) + \
                gaussian(x, NII_fl/3, NIIb_cent/1e4*(1+z), Nar_sig_wv)
                
    BLR_comp = gaussian(x,BLR_fl, Hal_cent/1e4*(1+z), BLR_sig_wv)
    
    spec = Cont+BLR_comp + Nar_comp + np.random.normal(0,np.random.uniform(0.1,0.3)*Nar_fl,2048)
                
    
    #plt.plot(x, spec, drawstyle='steps-mid')
    #plt.plot(x, BLR_comp)
    #plt.plot(x, Cont*np.ones_like(BLR_comp)+BLR_comp)
    
    Params = {'z':z}
    Params['Nar_fwhm'] = Nar_sig*2.35
    Params['BLR_fwhm'] = BLR_sig*2.35
    
    Params['BLR_fl'] = BLR_fl
    Params['Nar_fl'] = Nar_fl
    Params['NII_fl'] = NII_fl
    
    Params['Cont'] = Cont
    Params['Type'] = Type
    
    save = {'wave': x}
    save['flux'] = spec
    save['Params'] = Params
    
    
    return save


def spec_gen_Halpha_gen2():
    # generating random properties
    
    z = np.random.uniform(1.4, 2.5)
    while (z > 1.83) and (z < 1.94):
        z = np.random.uniform(1.4, 2.5)
    
    ## 1/2.35 is a conversion between fullwidth half max and the gaussian sigma...
    
    Nar_sig = np.random.uniform(200,700)/2.35   # Width of the narrow components
    Nar_fl = 10**np.random.uniform(-15.3, -12.3) # Flux of narrow Halpha
    NII_fl = np.random.uniform(0.1,1)*Nar_fl # Flux of [NII]
    
    Nar_sig_wv = Nar_sig/3e5*(Hal_cent/1e4*(1+z))

    BLR_sig = np.random.uniform(4000,8000)/2.35  
    BLR_sig_wv = BLR_sig/3e5*(Hal_cent/1e4*(1+z))
        
    # Are we doing Type 1 or 2
    Type = choice([0,1])
    Outflow= choice([0,1])
    
    # Generating BLR properties
    # It goes up to two because why the hell not (bright quasars exist)
    BLR_sig = np.random.uniform(4000,8000)/2.35 
    BLR_fl = np.random.uniform(0.1,2)*Nar_fl*Type   ## Note: multiplying by type sets the flux to 0, for type 0.
        
    ###
    # Selecting which band it was observed based on redshift
    Hw = np.linspace(1.42533, 1.86678, 2048)
    Kw = np.linspace(1.92543, 2.49972, 2048)
    
    if z<1.83:
        x=Hw
    elif z>1.94:
        x=Kw
    
    # More complex contiuum - I kind of made it up until it looked like something I say before - Need to be more scientific
    Cont_lvl = np.random.uniform(0.01,0.2)*Nar_fl
    Cont = ( (x-Hal_cent/1e4*(1+z))*np.random.uniform(-0.1, 0.1))*Cont_lvl*20+ Cont_lvl
      
    
    Nar_comp = gaussian(x, Nar_fl, Hal_cent/1e4*(1+z), Nar_sig_wv) + \
                gaussian(x, NII_fl, NIIr_cent/1e4*(1+z), Nar_sig_wv) + \
                gaussian(x, NII_fl/3, NIIb_cent/1e4*(1+z), Nar_sig_wv)
                
    Out_comp = gaussian(x, Hal_out_fl, Hal_cent/1e4*(1+z)+Hal_out_off_wv, Hal_out_sig_wv) + \
                gaussian(x, NII_out_fl, NIIr_cent/1e4*(1+z)+Hal_out_off_wv, Hal_out_sig_wv) + \
                gaussian(x, NII_out_fl/3, NIIb_cent/1e4*(1+z)+Hal_out_off_wv, Hal_out_sig_wv)       
                
    BLR_vel_offset = np.random.uniform(-400,400)    
    BLR_wv_offset = BLR_vel_offset/3e5 * Hal_cent/1e4*(1+z)
    BLR_comp = gaussian(x,BLR_fl, Hal_cent/1e4*(1+z)+BLR_wv_offset, BLR_sig_wv)
    
    noise_level = np.random.uniform(0.05,0.3)*Nar_fl
    # Method to the madness: we assume there was a 3-10sigma detection, which means the nar_fl = (3-20)*noise.
    spec = Cont + BLR_comp + Nar_comp + Out_comp + np.random.normal(0,noise_level,2048)
                
    
    plt.plot(x, spec, drawstyle='steps-mid')
    plt.plot(x, BLR_comp)
    plt.plot(x, BLR_comp+Out_comp)
    plt.plot(x, Cont*np.ones_like(BLR_comp)+BLR_comp)
    
    Params = {'z':z}
    Params['Nar_fwhm'] = Nar_sig*2.35
    Params['BLR_fwhm'] = Hal_out_sig*2.35
    Params['Out_fwhm'] = BLR_sig*2.35
    
    Params['BLR_fl'] = BLR_fl
    Params['Nar_fl'] = Nar_fl
    Params['NII_fl'] = NII_fl
    
    Params['Hal_out_fl'] = Hal_out_fl
    Params['NII_out_fl'] = NII_out_fl
    
    Params['Out_off'] = Hal_out_off
    
    Params['Cont'] = Cont_lvl
    Params['Type'] = Type
    
    save = {'wave': x}
    save['flux'] = spec
    save['Params'] = Params
    
    
    return save

In [35]:
Spec_info = spec_gen_Halpha()

In [36]:
save_obj(Spec_info, 'Spec_1')

In [37]:
Spec_info_ld = load_obj('Spec_1')

In [5]:
x = Spec_info_ld['wave']
spec = Spec_info_ld['flux']

plt.figure()

plt.plot(x, spec, drawstyle='steps-mid')